# DQN Implmentation

### Installations

In [1]:
# # Install environment and agent
# !pip install highway-env
# !pip install --upgrade sympy torch


### Learning using existing model

The following is the pesudocode that will be followed when creating the DQN

Useful: https://www.youtube.com/watch?v=RVMpm86equc&list=PL58zEckBH8fCMIVzQCRSZVPUp3ZAVagWi&index=2

https://github.com/saashanair/rl-series/tree/master/dqn

https://github.com/johnnycode8/gym_solutions/blob/main/frozen_lake_dql.py

<img src="DQN.png" style="width: 900px;" align="left"/>


Potential Problems: https://www.reddit.com/r/reinforcementlearning/comments/1555wgi/dqn_loss_increasing_and_rewards_decreasing/


For CNN:

https://www.reddit.com/r/MachineLearning/comments/3l5qu7/rules_of_thumb_for_cnn_architectures/


In [2]:
import numpy as np
class SumTree:
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros( 2 * capacity - 1 )
        self.data = np.zeros( capacity, dtype=object )
        self.writer = 0

    # propagate upwards to update the sum values
    def _propagate(self, index, change):
        parent = (index - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    # get the leaf nodes (Transaction)
    def _retrieve(self, index, s):
        left = 2 * index + 1
        right = left + 1

        if left >= len(self.tree):
            return index

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s-self.tree[left])

    def total(self):
        return self.tree[0]

    def add(self, p, data):
        index = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(index, p)

        self.write += 1
        # circular
        if self.write >= self.capacity:
            self.write = 0

    def update(self, index, p):
        change = p - self.tree[index]

        self.tree[index] = p
        self._propagate(index, change)

    def get(self, s):
        index = self._retrieve(0, s)
        data_index = index - self.capacity + 1

        return (index, self.tree[index], self.data[data_index])

In [236]:
import gymnasium as gym
import highway_env
import numpy as np
import random
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import torch.nn.init as init


# Define model
class MLPNetwork(nn.Module):
    def __init__(self, in_states, h1_nodes, out_actions):
        super(MLPNetwork, self).__init__()

        # Define network layers
        self.fc1 = nn.Linear(in_states, h1_nodes)   # first fully connected layer
        self.out = nn.Linear(h1_nodes, out_actions) # output layer
        self.out2 = nn.Linear(out_actions, 1) # output layer
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.relu(self.fc1(x)) # Apply rectified linear unit (ReLU) activation
        x = F.relu(self.out(x))         
        x = self.out2(x)
        x = self.softmax(x)
        return x

class CNN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CNN, self).__init__()
        # greyscale Image is(stack,height,width)
        stack, height, width = input_shape
        self.conv = nn.Sequential(
            nn.Conv2d(stack,16,kernel_size=4, stride=2, padding=1),
            nn.ReLU(),

            nn.Conv2d(16,32,kernel_size=3),
            nn.ReLU(),

            nn.Conv2d(32,64, kernel_size=2),
            nn.ReLU(),
        )
        
        # This is for finding the size to dense more robust compared to decision manually
        with torch.no_grad():
                # Torch uses(1,channels,height,width)
                test = torch.zeros(1, stack, height, width)
                find_conv_size = self.conv(test)
                conv_size = find_conv_size.numel()
                
        self.out1 = nn.Linear(conv_size,num_actions)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,x):
        x = self.conv(x)
        x = torch.flatten(x, start_dim=1)
        x = self.out1(x)
        x = self.softmax(x)
        return x

In [4]:
# Define memory for Experience Replay
# TODO: Prioritize, n-steps

from collections.__init__ import namedtuple

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory():
    def __init__(self, config, transition_type=Transition):
        self.capacity = config.get("capacity", 1000)
        self.index = 0
        self.transition_type = transition_type
        self.discount = config.get("discount", 0.99)

        self.memory = []
        self.device = config.get("device", torch.device("cpu"))
        self.n_steps = config.get("n_steps", 2)
    
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
            self.index = len(self.memory) - 1
        elif len(self.memory) > self.capacity:
            self.memory = self.memory[:self.capacity]
        # Faster than append and pop
        self.memory[self.index] = self.transition_type(*args)
        
        self.index = (self.index+1)%self.capacity # for circular memory

    def sample(self, batch_size, collapsed=True):
        if batch_size >  len(self.memory):
            batch_size = len(self.memory)
            
        if self.n_steps <= 1:
            # Directly sample transitions
            memories = random.sample(self.memory, batch_size)
            return self.unwrap_transition(*memories)
        else:
            # Sample initial transition indexes
            indexes = random.sample(range(len(self.memory)), batch_size)
            # Get the batch of n-consecutive-transitions starting from sampled indexes
            all_transitions = [self.memory[i:i+self.n_steps] for i in indexes]
            
            memories = map(self.collapse_n_steps, all_transitions) if collapsed else all_transitions

            return self.unwrap_transition(*memories)
        
    def collapse_n_steps(self, transitions):
        state, action, next_state, reward, done = transitions[0]
        discount = self.discount
        for transition in transitions[1:]:
            if done:
                break
            else:
                _, _, next_state, reward, done = transition
                discount *= self.discount
                reward += discount * reward
        return state, action, next_state, reward, done
    
    def unwrap_transition(self, *transition):
        state, action, next_state, reward, done = zip(*transition)
        
        states = torch.from_numpy(np.array(state)).float().to(self.device)
        actions = torch.from_numpy(np.array(action)).to(self.device)
        next_states = torch.from_numpy(np.array(next_state)).float().to(self.device)
        rewards = torch.from_numpy(np.array(reward)).float().to(self.device)
        dones = torch.from_numpy(np.array(done)).to(self.device)

        return states, actions, next_states, rewards, dones 

class PrioritizedReplayMemory(ReplayMemory):
    def __init__(self, config, transition_type=Transition):
        super().__init__(config)

        self.capacity = config.get("capacity", 1000)
        self.tree = SumTree(self.capacity)
        self.index = 0
        self.transition_type = transition_type

        self.memory = []
        self.device = config.get("device", torch.device("cpu"))
        
        self.alpha = config.get("alpha", 0.6)
        self.beta = config.get("beta", 0.2) #  will go to 1
        self.max_priority = 1  # priority for new samples, init as eps
        
    def store(self, *args):
        super().store(*args)
        self.tree.add(self.max_priority ** self.alpha, )
    
    def sample(self, batch_size, collapsed=True):
        priorities = torch.empty(batch_size, 1, dtype=torch.float)


In [5]:
from torch.utils.tensorboard import SummaryWriter
import os

class Metrics:
    def __init__(self, policy, result_file_name, use_metrics, time):
        self.use_metrics = use_metrics
        if not self.use_metrics:
            return
        new_num = str(len(os.listdir("./" +result_file_name)) + 1)
        file_name = f'{result_file_name}/{policy}_DQN_{new_num}_{time}'
        self.writer = SummaryWriter(log_dir=file_name, flush_secs=60)
            
    def add(self, type, y, x):
        if not self.use_metrics:
            return
        self.writer.add_scalar(type, y, x)
    def close(self):
        if not self.use_metrics:
            return
        self.writer.close()

In [226]:
import gymnasium as gym
import highway_env
import numpy as np
import random
import torch
import torch.optim as optim
import os
import datetime
from tqdm import tqdm
import json
import time

class DQNAgent:
    def __init__(self, params):
        self.q_net = {}
        self.q_target_net = {} 
        self.optimizer= {}        
        
        self.policy = params.get("policy", "CnnPolicy")        
        self.episode_num = params.get("episode_num", 10)

        self.epsilon = params.get("epsilon_max ", 1)
        self.epsilon_min = params.get("epsilon_min ", 0.1)
        self.epsilon_decay = params.get("epsilon_decay", 0.995)
        
        self.learning_rate = params.get("learning_rate", 5e-4)
        self.discount = params.get("discount", 0.2)
        self.batch_size = params.get("batch_size", 32)
        self.device = params.get("device", torch.device("cpu"))
        
        self.memory_capacity = params.get("memory_capacity", 1000)
        self.memory = {} # this is the memory buffer -> setting a limit
        self.n_steps = params.get("n_steps", 2)
        self.prioritize_memory = params.get("prioritize_memory", False)
        
        self.double = params.get("double", False)
        
        self.timeout = params.get("timeout_minute", 0) * 60 # in minutes
        ct = datetime.datetime.now()
        self.time = str(ct).replace(" ", "|")
        self.to_save_model = params.get("save_model", False)
        
        use_metrics = params.get("use_metrics", False)
        if use_metrics:
            self.create_folder("training_results")
            # self.save_params(params)
            
        self.metrics = Metrics(self.policy, "training_results", use_metrics, self.time)
        
    def initialize_weights(self, m):
        if isinstance(m, nn.Conv2d):
            # Xavier initialization for Conv2d weights
            init.xavier_uniform_(m.weight)
            init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):
            # Xavier initialization for Linear weights
            init.xavier_uniform_(m.weight)
            init.zeros_(m.bias)

    def create_network(self, env):
        if self.policy == "CnnPolicy":
            self.create_CNN(env)
        
        if self.policy == "MlpPolicy":
            self.create_MLP_Network(env)
        
        self.q_net.apply(self.initialize_weights)    
        self.update_target_network()
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=self.learning_rate)
        
    
    def create_CNN(self, env):
        self.num_states = env.observation_space.shape
        self.num_actions = env.action_space.n

        self.q_net = CNN(self.num_states, self.num_actions).to(self.device)
        self.q_target_net = CNN(self.num_states, self.num_actions).to(self.device)
        
    
    def create_MLP_Network(self, env):
        # the lanes
        self.num_states = env.observation_space.shape[1]
        self.num_actions = env.action_space.n

        self.q_net = MLPNetwork(self.num_states, self.num_states, self.num_actions).to(self.device)
        self.q_target_net = MLPNetwork(self.num_states, self.num_states, self.num_actions).to(self.device)
    
    def update_target_network(self):
        self.q_target_net.load_state_dict(self.q_net.state_dict())
    
    def learn(self, env):
        self.create_network(env)
        
        # if (self.prioritize_memory):
        self.memory = ReplayMemory({
            "capacity": self.memory_capacity,
            "device": self.device,
            "n_steps": self.n_steps,
        })

        self.prefill_memory(env, self.batch_size)

        start_time = time.time()        
        
        for epoch in tqdm(range(self.episode_num), desc="Training Model"):
            state = env.reset()[0]
            
            # True when agent reaches the end states (colliding or passing the time)
            done = False 
            
            # TODO: see how many actions until truncate
            # True when agent takes more than some actions 
            truncated = False
            episode_rewards = []
            episode_loss = []
            episode_len = 0
            while(not done and not truncated):
                # choose best action
                action = self.get_action(state)
                next_state, reward, done, truncated, _ = env.step(action)
                self.memory.store(state, action, next_state, reward, done)
                
                episode_loss.append(self.experience_replay())
                
                state = next_state
                
                episode_rewards.append(reward)
                episode_len += 1
                
            self.metrics.add("rollout/rewards", sum(episode_rewards) / len(episode_rewards), epoch)
            self.metrics.add("rollout/exploration-rate", self.epsilon, epoch)
            self.metrics.add("rollout/episode-length", episode_len, epoch)
            self.metrics.add("train/loss", sum(episode_loss) / len(episode_loss), epoch)
            
            if self.timeout:
                elapsed_time = time.time() - start_time
                if elapsed_time > self.timeout:
                    print("Timeout reached. Stopping training.\n")
                    break
            
            if epoch % 10 == 0:
                self.decay_epsilon()
                self.update_target_network()
            
        self.metrics.close()
        
        if self.to_save_model:
            self.save_model()
        
    # either the policies are able to get miltuple actions and into the NN or the input of NN should be able to handle all of these
    # output (one of): {0: 'LANE_LEFT', 1: 'IDLE', 2: 'LANE_RIGHT', 3: 'FASTER', 4: 'SLOWER'}
    def get_action(self, state, eval_mode=False):
        if random.random() <= self.epsilon and not eval_mode: # amount of exploration reduces with the epsilon value
            return random.randrange(self.num_actions)
        
        state = torch.tensor(np.array([state]), dtype=torch.float32).to(self.device)
        actions = self.q_net(state)
        return torch.argmax(actions).item()             

    def experience_replay(self):
        states, actions, next_states, rewards, dones = self.memory.sample(self.batch_size)
        
        q_pred = self.q_net(states)
        # q value of the action taken
        q_pred = q_pred.gather(1, actions.view(-1, 1)) 
        q_pred = q_pred.squeeze(1)

        # Double DQN
        if self.double:
             # pick best actions from policy network
            q_best_action = self.q_net(next_states)
            _, q_best_action = q_best_action.max(dim=1)
            q_best_action = q_best_action.unsqueeze(1)
            
            # use those actions for the target policy
            q_target = self.q_target_net(next_states)
            q_target = q_target.gather(1, q_best_action)
            q_target = q_target.squeeze(1)
        else:
            q_target = self.q_target_net(next_states)
            q_target = q_target.max(dim=1).values
        
        # setting Q(s',a') to 0 when the current state is a terminal state
        q_target[dones] = 0.0
        
        y_j = rewards + (self.discount * q_target)
        
        # calculate the loss as the mean-squared error of yj and qpred
        self.optimizer.zero_grad()
        loss = F.mse_loss(y_j, q_pred).mean()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
        
    def prefill_memory(self, env, prefill_num):
        for _ in tqdm(range(prefill_num), desc="Prefilling Memory "):
            done = False
            truncated = False
            state = env.reset()[0]

            while not done and not truncated:
                action = env.action_space.sample()
                next_state, reward, done, truncated, info = env.step(action)
                self.memory.store(state, action, next_state, reward, done)    
                
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon*self.epsilon_decay)
    
    def evaluate(self, env, episode_num):
        # add camera here
        for _ in tqdm(range(episode_num), desc="Evaluating Model"):
            state = env.reset()[0]  
            done = False      
            truncated = False 

            # Agent navigates map until it falls into a hole (terminated), reaches goal (terminated), or has taken 200 actions (truncated).
            while(not done and not truncated):  
                # Select best action   
                action = self.get_action(state, eval_mode=True)
                next_state, reward, done, truncated, info = env.step(action)
                state = next_state
                env.render()
        
    def save_model(self):
        folder_name = self.policy + "_save_models"
        self.create_folder(folder_name)
        new_model_num = str(len(os.listdir("./" +folder_name)) + 1)
        file_name = f'{folder_name}/DQN_{new_model_num}_{self.time}.pth'
        state = {'state_dict': self.q_net.state_dict(),
            'optimizer': self.optimizer.state_dict()}
        torch.save(state, file_name)
        
    def load_model(self, env, file_name):
        folder_name = self.policy + "_save_models"
                
        filename = folder_name + "/" + file_name + ".pth"
        self.create_network(env)
        
        models = torch.load(filename, map_location=self.device)
        
        self.q_net.load_state_dict(models['state_dict'])
        self.optimizer.load_state_dict(models['optimizer'])
    
    def save_params(self, params):
        folder_name = "hyperparameters"
        self.create_folder(folder_name)
        
        file_name = f'./{folder_name}/{self.policy}_DQN_{self.time}'
        with open(file_name + '.txt', 'w') as file:
            file.write(json.dumps(str(params)))

    def create_folder(self, directory_name):
        try:
            os.mkdir(directory_name)
            print(f"Directory '{directory_name}' created successfully.")
        except FileExistsError:
            return
        except PermissionError:
            print(f"Permission denied: Unable to create '{directory_name}'.")
        except Exception as e:
            print(f"An error occurred: {e}")


In [190]:
config = {}
policy = "CnnPolicy"
# policy = "MlpPolicy"

if policy == "CnnPolicy":
    config={
        "lanes_count" : 3,
        "observation": {
            "type": "GrayscaleObservation",
            "observation_shape": (128, 64),
            "stack_size": 4,
            "weights": [0.2989, 0.5870, 0.1140],  # weights for RGB conversion keep this conversion this is in the highway env page
            "scaling": 1.75,
        },
    }
else:
    config = {
        "lanes_count" : 3,
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 5,
            "features": ["presence", "x", "y", "vx", "vy", "cos_h", "sin_h"],
            "features_range": {
                "x": [-100, 100],
                "y": [-100, 100],
                "vx": [-20, 20],
                "vy": [-20, 20]
            },
            "absolute": False,
            "order": "sorted"
        }
    }

In [211]:
import torch
params = {
    'policy' : policy,
    'episode_num' : 10000,
    'discount' : 0.7,
    'batch_size' : 16,
    'learning_rate': 5e-6,
    'n_steps': 4,
    'double': True,
    'device' : torch.device("mps"),
    'memory_capacity' : 10000,
    'timeout_minute': 15,
    'use_metrics' : True,
    'save_model': True,
}

for seed in range(3):
    torch.manual_seed(seed)
    dqn_agent = DQNAgent(params)
    env = gym.make('highway-fast-v0', render_mode='rgb_array', config=config)
    dqn_agent.learn(env)

# env = gym.make('highway-v0', render_mode='rgb_array', config=config)
# dqn_agent.evaluate(env, 5)

# if you wanna save a model again
# dqn_agent.save_model("highway_dqn_model")

Training Model:  15%|█▌        | 1523/10000 [15:00<1:23:30,  1.69it/s]


Timeout reached. Stopping training.



Training Model:  14%|█▍        | 1448/10000 [15:00<1:28:37,  1.61it/s]


Timeout reached. Stopping training.



Training Model:  15%|█▍        | 1473/10000 [15:00<1:26:51,  1.64it/s]

Timeout reached. Stopping training.



In [237]:

env = gym.make('highway-v0', render_mode='rgb_array', config=config)

dqn_agent_test = DQNAgent(params)
dqn_agent_test.load_model(env, "DQN_4_2024-12-24|14:11:48.566160")

dqn_agent_test.evaluate(env, 20)


/var/folders/ll/m_r158w50cqfn3ydymrpm7rc0000gn/T/ipykernel_63725/1993401512.py:240: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  models = torch.load(filename, map_location=

tensor([[0.0185, 0.2510, 0.2137, 0.1976, 0.3192]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.4049, 0.4373, 0.0794, 0.0192, 0.0591]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0151, 0.8696, 0.0988, 0.0057, 0.0108]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1319, 0.3868, 0.3957, 0.0533, 0.0323]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0477, 0.3249, 0.4347, 0.0774, 0.1153]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0148, 0.0608, 0.6403, 0.0468, 0.2372]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0093, 0.0891, 0.6170, 0.2387, 0.0459]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0201, 0.3056, 0.4747, 0.1253, 0.0744]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0129, 0.6341, 0.1415, 0.0842, 0.1273]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0917, 0.2915, 0.3395, 0.1494, 0.1278]], device='mps:0',
       grad_fn=

Evaluating Model:   5%|▌         | 1/20 [00:04<01:30,  4.77s/it]

tensor([[0.0221, 0.3604, 0.4267, 0.0916, 0.0992]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.3926, 0.0366, 0.5376, 0.0075, 0.0257]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0572, 0.3040, 0.4567, 0.1623, 0.0198]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0435, 0.3207, 0.0895, 0.2204, 0.3258]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0789, 0.3173, 0.3057, 0.2595, 0.0386]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0073, 0.4538, 0.3775, 0.1127, 0.0487]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0149, 0.1094, 0.1438, 0.6891, 0.0428]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0276, 0.6583, 0.0192, 0.0200, 0.2750]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0413, 0.0296, 0.3465, 0.5556, 0.0270]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0878, 0.0422, 0.3482, 0.1348, 0.3870]], device='mps:0',
       grad_fn=

Evaluating Model:  10%|█         | 2/20 [00:14<02:22,  7.90s/it]

...

Evaluating Model:  90%|█████████ | 18/20 [01:14<00:04,  2.31s/it]

tensor([[0.0307, 0.1646, 0.5378, 0.2169, 0.0500]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.4875, 0.0699, 0.3521, 0.0706, 0.0199]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1435, 0.4581, 0.3153, 0.0010, 0.0821]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0103, 0.2525, 0.5590, 0.1259, 0.0523]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0593, 0.0395, 0.1893, 0.2003, 0.5115]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0472, 0.0711, 0.1127, 0.3710, 0.3979]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0126, 0.1423, 0.6838, 0.0652, 0.0961]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0024, 0.2011, 0.6734, 0.0880, 0.0351]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1488, 0.2993, 0.4740, 0.0458, 0.0322]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.2306, 0.4885, 0.2316, 0.0243, 0.0250]], device='mps:0',
       grad_fn=

Evaluating Model:  95%|█████████▌| 19/20 [01:24<00:04,  4.68s/it]

tensor([[0.0850, 0.2078, 0.2198, 0.2026, 0.2847]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0679, 0.6665, 0.1065, 0.0285, 0.1306]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0107, 0.6353, 0.2254, 0.0317, 0.0969]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1768, 0.3929, 0.1705, 0.2140, 0.0457]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.1787, 0.1172, 0.0955, 0.5234, 0.0852]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0756, 0.1094, 0.5357, 0.1828, 0.0964]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0094, 0.1014, 0.8182, 0.0552, 0.0158]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.2297, 0.0324, 0.6684, 0.0154, 0.0542]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.2438, 0.0510, 0.1469, 0.0014, 0.5570]], device='mps:0',
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0069, 0.0441, 0.2712, 0.0222, 0.6556]], device='mps:0',
       grad_fn=

Evaluating Model: 100%|██████████| 20/20 [01:32<00:00,  4.60s/it]


### Run the Tensorboard

In [8]:
%reload_ext tensorboard

%tensorboard --logdir training_results --host localhost --port 6010